In [ ]:
import sys
sys.path.append('P:\\python\\_functions\\')
from toImportDevFunctions import *

In [ ]:
import pypdf
from pypdf import PdfReader, PdfWriter, PdfMerger, PageObject, Transformation, PaperSize
pypdf.__version__

In [ ]:
#Setup inputs
# pdf to work with
fPDF = 'test.pdf'
print(os.path.exists(fPDF))

#mid margin shift (for binding)
margin = 20 

#number of papers for signature
sigThick = 2

#blank start and blank end pages (extend number of pgaes so (pdfPgNum % (sigThick*4)) == 0 --next cell helps)
blankStart = 0
blankEnd = 0

In [ ]:
reader = PdfReader(fPDF)

pdfPgNum = len(reader.pages)
pgTotal = (blankStart + blankEnd + pdfPgNum)
sheetsNeeded = int(np.ceil((pdfPgNum / 4)))

if (pgTotal % (4*sigThick)!=0):
    print('Fixit:', pgTotal, 'to add:', (4*sigThick)-(pgTotal % (4*sigThick)))
else:
    print('OK:', pgTotal , sheetsNeeded)

In [ ]:
reader = PdfReader(fPDF)

if (blankStart!=0)|(blankEnd!=0):
    print('Save and reload')
    writer = PdfWriter()

    for i in range(blankStart):
        writer.add_blank_page(width=PaperSize.A4.width, height=PaperSize.A4.height)
    
    for i in range(pdfPgNum):
        writer.add_page(reader.pages[i])
    
    for i in range(blankEnd):
        writer.add_blank_page()
    
    with open('tmp.pdf', "wb") as fp:
        writer.write(fp)

    reader = PdfReader('tmp.pdf')
    pdfPgNum = len(reader.pages)

pdfPgNum

In [ ]:
#create list for rearrange
lPgs = list(range(pdfPgNum))
lPgsReOrd = []
for i in range(len(lPgs)//(sigThick*4)):
    iSt = i*sigThick*4
    iEnd= iSt + sigThick*4

    lSigSt = lPgs[iSt:iEnd]
    lSigEnd = []
    for i in range(sigThick):
        lSigEnd.append(lSigSt.pop(-1))
        lSigEnd.append(lSigSt.pop(0))
        lSigEnd.append(lSigSt.pop(0))
        lSigEnd.append(lSigSt.pop(-1))

    lPgsReOrd = lPgsReOrd+lSigEnd
    print(lSigEnd)
    
#lPgsReOrd

In [ ]:
#rearrange pages
writer = PdfWriter()

for i in lPgsReOrd:
    page = reader.pages[i]
    writer.add_page(page)

with open(fPDF[:-4]+'_reord.pdf', "wb") as fp:
        writer.write(fp)

In [ ]:
#create 2 pager Signatures with additional middle margin
writer = PdfWriter()
for iter in range(0, len(reader.pages)-1, 2):
    print(iter)

    destpage = writer.add_blank_page(width=PaperSize.A3.height, height=PaperSize.A3.width)
    
    lhs = reader.pages[iter]
    rhs = reader.pages[iter+1]

    destpage.merge_transformed_page(lhs, Transformation().translate(-1*margin,0,),)
    destpage.merge_transformed_page(rhs, Transformation().translate(margin+PaperSize.A4.width,0,),)


with open(fPDF[:-4]+'_toPrint_py.pdf', "wb") as fp:
    writer.write(fp)

In [ ]:
#reader = PdfReader(fPDF[:-4]+'_2up_py.pdf')
#print(fPDF[:-4]+'_2up_py.pdf')
#box = reader.pages[0].mediabox
#print(box)
#print(box.width)
#print(box.height)